In [100]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import boto3
from botocore.exceptions import ClientError
import os
import json
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [37]:
twitter_data = pd.read_csv("Corona_NLP_train.csv", encoding = "ISO-8859-1")

In [38]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Для начала немного преподготовим наши данные, и объеденим Extremely Negative с Negative  
и Extremely Positive с Positive

In [39]:
for i in range(0, len(twitter_data)):
    if(twitter_data['Sentiment'][i] == 'Extremely Negative'):
        twitter_data['Sentiment'][i] = 'Negative'
    elif(twitter_data['Sentiment'][i] == 'Extremely Positive'):
        twitter_data['Sentiment'][i] = 'Positive'

<ipython-input-39-9eae80cfb354>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['Sentiment'][i]='Negative'
<ipython-input-39-9eae80cfb354>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['Sentiment'][i]='Positive'


Теперь очистим данные от хэштегов, ссылок и прочего

In [40]:
def clean(data):
    out = []
    for i in range(0, len(data)):
        temp = data[i]
        temp = re.sub("@\S+", " ", temp)
        temp = re.sub("https*\S+", " ", temp)
        temp = re.sub("#\S+", " ", temp)
        temp = re.sub("\'\w+", '', temp)
        temp = re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
        temp = re.sub(r'\w*\d+\w*', '', temp)
        temp = re.sub('\s{2,}', " ", temp)
        list_of_temp = temp.split(" ")
        list_of_temp = list(map(lambda x: x.lower(), list_of_temp))
        temp = ' '.join(list_of_temp)
        out.append(temp)
    return out

In [41]:
def replace_data(out):
    twitter_data.drop(['OriginalTweet'], axis=1)
    twitter_data['OriginalTweet'] = out

In [42]:
out = clean(list(twitter_data['OriginalTweet']))

In [43]:
replace_data(out)

In [44]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,and and,Neutral
1,3800,48752,UK,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,NaN,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,me ready to go at supermarket during the outbr...,Negative


Теперь поделим наш датасет на классы

In [45]:
list_of_labels = list(twitter_data['Sentiment'].unique())
list_of_labels

['Neutral', 'Positive', 'Negative']

In [46]:
def get_twitts(label, filename):
    l = twitter_data[twitter_data.Sentiment == label]
    l = list(l['OriginalTweet'])
    
    f = open(f"{filename}.txt", "w")
    for twitt in l[0:int(len(l)/10)]:
        f.write(twitt + "\n")
    f.close()

In [47]:
for label in list_of_labels:
    get_twitts(label, label)

In [113]:
class ComprehendDetect:
    def __init__(self, comprehend_client):
        self.comprehend_client = comprehend_client

    def detect_key_phrases(self, text, language_code):
        try:
            response = self.comprehend_client.detect_key_phrases(
                Text=text, LanguageCode=language_code)
            phrases = response['KeyPhrases']
        except ClientError:
            raise
        else:
            return phrases

    def detect_sentiment(self, text, language_code):
        try:
            response = self.comprehend_client.detect_sentiment(
                Text=text, LanguageCode=language_code)
        except ClientError:
            raise
        else:
            return response
        
access_key_id = os.getenv('MY_ACCESS_KEY_ID')
secret_access_key = os.getenv('MY_SECRET_ACCESS_KEY')
comp = boto3.client('comprehend', region_name="us-east-1", aws_session_token = "IQoJb3JpZ2luX2VjEN3//////////wEaCXVzLXdlc3QtMiJIMEYCIQDwDaa3iRyHxfv1/17JkOINlWpmcYr8v0T89LUbq+af+gIhALkPo+Zjl6fiPWArPkfYHs3la8SqhhsKRZpC7X4eMacgKrQCCGYQABoMNjEzMjg5NjM4NDE4Igxk84kQHvp/EPLx0YQqkQKEgMceKtfS+4ir5IrcKS5P1OHWkacRYULVcjJ5jiHI0cBdzc1IiCPoTtHrA2tvHPjbevT5d5SAwDqCKSoruYradaJJ9jQa94BkfUGaoXlGLDJETg/wvS9d4Ae+vyZiMN/+FjmB7DoTRlWbxk/Z/+bnWJrUXptHv2xCNPqowUsGyPdSbe2NYj1tpBXjk7SPu1przhlOinnMaQQWPrMKLmhFrhrisQLi0rUPLsUOkK6hEfRATEuO0yLPN29DCzpqhheRNINxwP99yv3qV3dLsREQj3umsuH65iLV9W1ECM3tGNgGXsfaGKbMGSM89wk3OFCWa0KP/la66TYSHmuMg/M3d7J8p+cohTIvfa9AbkgnYUQwk+zbhAY6nAEaU9q43L2UGV0hQrDqmk11anPqSK5QH1G9KdsVpGlLPmnvrxIeGIVfeRV3qrdxrZyjOlrk+B9/mymhNSr5z6uM8QNhfC5spHQXb67wUVmjdHWFf55FpcwDuH+a4xsVHBSutN9k4kP2gGeD4ttgCoAA7XdOqaIbb7Yjk5iN0r48G1lUEPue4vvpWgHE/ax3mW7IW0UwgdR0hEPbIsc=")

comp_detect = ComprehendDetect(comp)

phrases = comp_detect.detect_key_phrases('', 'en')

In [67]:
def get_accuracy(filename, clean=False):
    jsons = []
    with open(filename) as f:
        for line in f:
            jsons.append(json.loads(line))
    
    score = 0
    for j in jsons:
        original_sentiment = j['File'].split(".")[0].lower()
        if clean:
            original_sentiment = original_sentiment.split("_")[0]
        pred_sentiment = j['Sentiment'].lower()
        if original_sentiment == pred_sentiment:
            score += 1
                        
    return score / len(jsons) * 100

Как мы видим с помощью амазон сервиса мы получили крайне низкий результат accuracy  
Лишь немного выше чем рандомный выбор с вероятностью угадать 1/3

In [141]:
accuracy = get_accuracy("sentiment_from_comprehend.txt")
print(f"Accuracy = {accuracy}%")

Accuracy = 39.499270782693245%


Теперь попробуем всё тоже самое, только удалим стоп-слова и преобразуем все слова в начальную форму

In [17]:
## если не скачено
nltk.download('stopwords')

In [50]:
list_of_twitts = list(twitter_data['OriginalTweet'])
out_list = []

morph = pymorphy2.MorphAnalyzer(lang='uk')

for twitt in list_of_twitts:
    # удаляем стоп-слова
    filtered_words = [word for word in twitt.split(" ") if word not in stopwords.words('english')]
    # преобразуем в начальную форму
    normal_form_words = []
    for word in filtered_words:
        normal_form_words.append(morph.parse(word)[0].normal_form)
    # объединяем
    out_list.append(' '.join(normal_form_words))

In [51]:
# заменяем данные на очищенные
replace_data(out_list)

In [52]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,Neutral
1,3800,48752,UK,16-03-2020,advice talk neighbours family exchange phone n...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths give elderly ...,Positive
3,3802,48754,NaN,16-03-2020,food stock one empty please panic enough food ...,Positive
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Negative


In [53]:
for label in list_of_labels:
    get_twitts(label, label + '_cleanup')

Как можно наблюдать точность уменьшилась, что меня несколько удивило

In [140]:
accuracy = get_accuracy("sentiment_from_comprehend_clean_data.txt", True)
print(f"Accuracy = {accuracy}%")

Accuracy = 31.40495867768595%


In [71]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data['OriginalTweet'], twitter_data['Sentiment'], test_size=0.2, random_state=42)

In [123]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)


train_data_features = vectorizer.fit_transform(list(twitter_data['OriginalTweet'])).toarray()

In [127]:
print(twitter_data.shape)
print(train_data_features.shape)

(41157, 6)
(41157, 5000)


In [128]:
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(train_data_features[:30000], list(twitter_data['Sentiment'])[:30000])

In [131]:
predictions = forest.predict(train_data_features[30000:])
print(predictions)

['Positive' 'Positive' 'Positive' ... 'Positive' 'Positive' 'Negative']


In [136]:
def get_accuracy_from_list(real, predictions):
    score = 0
    for i in range(len(real)):
        if real[i] == predictions[i]:
            score += 1
    return score / len(real) * 100

С помощью этого метада мы получили уже куда большую точность

In [139]:
accuracy = get_accuracy_from_list(list(twitter_data["Sentiment"])[30000:], predictions)
print(f"Accuracy = {accuracy}%")

Accuracy = 76.57972573272384%


Теперь попробуем воспользоваться методом word2vec